## Movielens 영화 추천
Movielens 데이터 implict 방식으로 추천 시스템 만들기

- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. 
  - MovieLens 1M Dataset 사용


- 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트할 수 있습니다.


- 별점을 **시청횟수**로 해석해서 생각하겠습니다.


- 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하겠습니다.

#### 데이터셋 다운로드 (Ubuntu) 
wget http://files.grouplens.org/datasets/movielens/ml-1m.zip


### 데이터 준비 및 전처리
- 3점 미만으로 준 데이터는 선호하지 않는다고 가정했으니, 3점 이상의 데이터만 남긴다.

- 메타 데이터 읽어오기, 컬럼명 정리 등의 작업 진행


In [2]:
# 모듈 임포트
import pandas as pd
import numpy as np

In [3]:
# 인덱싱은 이미 완료되어 있다.
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [4]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [5]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [6]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [7]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
movies["genre"]

0        Animation|Children's|Comedy
1       Adventure|Children's|Fantasy
2                     Comedy|Romance
3                       Comedy|Drama
4                             Comedy
                    ...             
3878                          Comedy
3879                           Drama
3880                           Drama
3881                           Drama
3882                  Drama|Thriller
Name: genre, Length: 3883, dtype: object

In [9]:
# 검색을 쉽게하기 위해 genre 문자열과 제목을 소문자로 바꿔줍시다.
movies['genre'] = movies['genre'].str.lower()
movies['title'] = movies['title'].str.lower()
movies.head(5)

,movie_id,title,genre
0,1,toy story (1995),animation|children's|comedy
1,2,jumanji (1995),adventure|children's|fantasy
2,3,grumpier old men (1995),comedy|romance
3,4,waiting to exhale (1995),comedy|drama
4,5,father of the bride part ii (1995),comedy


In [10]:
# 영화의 개수
len(movies)

3883

In [11]:
# 검색이 용이하게 title에 있는 (년도) 데이터 지우기
# 검색을 하기위한 pandas를 만들었다.
# 년도는 다르지만 같은 이름인 경우가 발생
# 대부분 같은 시리즈의 리메이크 요소이니 같은 영화라 생각하고 진행하겠다.

movies['title'] = movies['title'].str.replace(r"\([0-9]*\)"," ")
movies['title'] = movies['title'].str.strip()  # 공백이 있을 수 있으니 양끝 공백 제거
search_movies = movies[['movie_id', 'title']].copy()

search_movies

,movie_id,title
0,1,toy story
1,2,jumanji
2,3,grumpier old men
3,4,waiting to exhale
4,5,father of the bride part ii
...,...,...
3878,3948,meet the parents
3879,3949,requiem for a dream
3880,3950,tigerland
3881,3951,two family house


년도 정보를 제거하면서 같은 제목을 가진 영화가 남아 있게 되었다.


- 이름이 같은 정보는 나중에 내가 원하는 영화가 아닌 다른 데이터를 가져올 수 있고 같은 이름이면 대체로 비슷한 유형의 영화이니 같은 데이터를 처음 나온 데이터만 남기는 전처리를 진행한다.

In [12]:
search_movies[search_movies['title'].str.contains("titanic")]
# 년도 정보를 제거하면서 같은 제목을 가진 영화가 남아 있게 되었다.
# 이름이 같은 정보는 나중에 내가 원하는 영화가 아닌 다른 데이터를 가져올 수 있고
# 같은 이름이면 대체로 비슷한 유형의 영화이니 같은 데이터를 처음 나온 데이터만 남기는
# 전처리를 진행한다.

,movie_id,title
1672,1721,titanic
2088,2157,"chambermaid on the titanic, the"
3334,3403,raise the titanic
3335,3404,titanic


In [668]:
# 년도 데이터를 제거하기 전의 모습
movies[movies['title'].str.contains("titanic")]

,movie_id,title,genre
1672,1721,titanic (1997),drama|romance
2088,2157,"chambermaid on the titanic, the (1998)",romance
3334,3403,raise the titanic (1980),drama|thriller
3335,3404,titanic (1953),action|drama


### Pandas 중복 데이터 제거
- **pandas.DataFrame.drop_duplicates**

- pandas의 drop_duplicates는 중복된 값을 제거한 뒤 DataFrame을 반환합니다.

#### subset: 중복제거할 column 결정
  - DataFrame의 column 혹은 [column, column]으로 명시하면 명시된 column을 기준으로 중복제거가 일어납니다.
  - subset을 명시하지 않으면 전체 데이터 셋을 기준으로 중복제거를 합니다.
  
#### keep: 중복된 값에 대한 처리 
default  : **"first"**
  - first: 첫 번째 중복 값을 남기고, 이외 중복 값을 모두 제거합니다.
  - last: 마지막 번째 중복 값을 남기고, 이외 중복 값을 모두 제거합니다.
  - False: 중복된 값들을 모두 제거합니다.

#### inplace: 원본데이터 자체에서 진행하는지 새로운 DataFrame을 만들지 정하는boolean 값
default : **False**
  - True면 원본 DataFrame 자체에서 중복제거를 합니다. 반환 값은 None
  - False면 원본 DataFrame을 건들지 않고 중복제거한 새로운 DataFrame을 만들어 값을 반환합니다.
  

In [13]:
# 이름이 같은 데이터 제거시 3841개의 데이터가 남는다.
# 중복 데이터의 개수가 많지 않으니 제거해서 사용한다.

print("중복 제거 전 데이터 개수 : ", len(movies))
movies.drop_duplicates("title",inplace=True)
print("중복 제거 후 데이터 개수 : ", len(movies) )


중복 제거 전 데이터 개수 :  3883
중복 제거 후 데이터 개수 :  3841


In [14]:
# 중복 제거 데이터 확인
movies['title']

0                         toy story
1                           jumanji
2                  grumpier old men
3                 waiting to exhale
4       father of the bride part ii
                   ...             
3878               meet the parents
3879            requiem for a dream
3880                      tigerland
3881               two family house
3882                 contender, the
Name: title, Length: 3841, dtype: object

---
## 데이터 분석하기
- ratings에 있는 유니크한 영화 개수
- rating에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)

In [889]:
ratings.head(5)

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


### ratings에 있는 고유한 영화 개수

In [890]:
# 고유한 영화를 찾아내는 코드
unique_movies = movies["movie_id"].unique()

### ratings에 있는 고유한 사용자의 수

In [891]:
# 고유한 영화, 사용자를 찾아내는 코드
unique_users = ratings["user_id"].unique()

### 가장 인기 있는 영화 30개(인기순)
아래 2가지를 따로 기준 잡아서 인기순 30를 나열하겠습니다.

- 1. 총 리뷰의 평균 값을 기준으로 30개


- 2. 평점을 준 count의 수가 많은 순으로 30개


#### 평균을 기준으로 할때 표본 데이터의 부족 발생 문제  

위와 같이 하는 이유는 해당 영화를 봤지만 평점을 매긴 사람이 10명일때, 10명 모두 5점을 준 경우  
평균으로 했을때 5점 만점이 되어서 인기가 있다고 나타날 수 있다.  

그리고 1000명이 해당 영화의 점수를 줘서 평균 4.5가 된 영화를 생각해보면, 준수한 성적 이지만 위의 5.0보다는  
작기 때문에 인기가 덜 하다고 여길 수가 있다.

#### 총합을 기준으로 했을때 본 사람이 다른 영화보다 몇 배나 많지만 모두 1,2 점을 준 경우

위와 같은 경우에는 예를들어, A 영화의 총합이 1000이고 B 영화의 총합이 500인 경우 A영화가 더 인기가 있구나라고 생각할 수 있지만, 데이터를 봤을때 A는 1000명이 봐서 1점을 준 경우이고 B는 100명이 봐서 모두 5점을 준 경우라고 생각하면 얘기가 달라지게 된다.

<br>

- 위의 경우와 같이 평균으로만 인기를 판단하면 생기는 문제와 총합으로만 봤을때 생기는 문제가 있기에,  
위의 2가지 경우를 따로 기준 잡아 인기순으로 표현하겠습니다.

---
ratings 데이터와 movies 데이터를 movie_id를 기준으로 하나로 merge해 준다.
- 유저들이 해당 movie에 어떤 rating을 매겼는지 구하기 위해 합쳐준다.

In [48]:
user_ratings_movie = pd.merge(ratings[["user_id","movie_id", "count"]], movies[["movie_id","title"]], on="movie_id")
user_ratings_movie[:10]

,user_id,movie_id,count,title
0,1,1193,5,one flew over the cuckoo's nest
1,2,1193,5,one flew over the cuckoo's nest
2,12,1193,4,one flew over the cuckoo's nest
3,15,1193,4,one flew over the cuckoo's nest
4,17,1193,5,one flew over the cuckoo's nest
5,18,1193,4,one flew over the cuckoo's nest
6,19,1193,5,one flew over the cuckoo's nest
7,24,1193,5,one flew over the cuckoo's nest
8,28,1193,3,one flew over the cuckoo's nest
9,33,1193,5,one flew over the cuckoo's nest


#### 1. 총 리뷰의 평균 값을 기준으로 30개
위에서 merge한 두 데이터 프레임을 title를 기준으로 병합을 진행 합니다.
  - 같은 title일 경우 같은 movie_id를 가지고 있다.
그리고 각 title 별로 count를 묶어 주었고 묶은 count열을 mean으로 평균을 낸 후 
**내림차순**으로 30개의 데이터를 나타낸다.

In [44]:
group_ratings_movie = user_ratings_movie['count'].groupby(user_ratings_movie['title'])
group_ratings_movie.mean().sort_values( axis=0, ascending=False).head(30)

title
gate of heavenly peace, the                                     5.000000
criminal lovers (les amants criminels)                          5.000000
foreign student                                                 5.000000
country life                                                    5.000000
identification of a woman (identificazione di una donna)        5.000000
song of freedom                                                 5.000000
follow the bitch                                                5.000000
schlafes bruder (brother of sleep)                              5.000000
black sunday (la maschera del demonio)                          5.000000
one little indian                                               5.000000
paralyzing fear: the story of polio in america, a               5.000000
message to love: the isle of wight festival                     5.000000
late bloomers                                                   5.000000
ulysses (ulisse)                             

---
#### 2. 평점을 준 count의 수가 많은 순으로 30개
1.에서 한 절차와 같이 merge한 두 데이터 프레임을 title를 기준으로 병합을 진행 합니다.
  - 같은 title일 경우 같은 movie_id를 가지고 있다.
그리고 각 title 별로 count를 묶어 주었고 묶은 count열을 sum으로 총 합을 낸 후 
**내림차순**으로 30개의 데이터를 나타낸다.

In [45]:
group_ratings_movie = user_ratings_movie['count'].groupby(user_ratings_movie['title'])
group_ratings_movie.sum().sort_values( axis=0, ascending=False).head(30)

title
american beauty                                   14449
star wars: episode iv - a new hope                13178
star wars: episode v - the empire strikes back    12648
saving private ryan                               11348
star wars: episode vi - return of the jedi        11303
raiders of the lost ark                           11179
silence of the lambs, the                         11096
matrix, the                                       10903
sixth sense, the                                  10703
terminator 2: judgment day                        10513
fargo                                             10465
schindler's list                                  10317
braveheart                                        10125
shawshank redemption, the                         10085
back to the future                                10081
godfather, the                                     9965
princess bride, the                                9866
jurassic park                             

인기순 같은 경우 어떤 기준으로 할 지 판단해서 진행하는 거여서 어느 방법이 더 좋은지는 크게 의미가 없는것 같습니다. 2 방법 모두 각자의 장점과 단점이 있어서요.

- 그렇지만 위의 결과만 봤을때 group_ratings_movie.sum()의 결과로 나온 영화는 들어본 것들이 많지만(스타워크 터미네이터, 매트린스, 양들의 침묵 등) 평균으로 했을때는 대부분 모르는 영화이여서 현재 데이터 상으로 봤을때는 총 합을 기준으로 인기를 나타내는게 더 좋은것 같습니다. 

### 선호하는 영화를 골라서 rating에 추가하기
- 사용자 초기 정보 세팅
- 내가 선호하는 영화 5가지를 추가하여 모델 검증에 사용한다.

In [893]:
movies.head(5)

,movie_id,title,genre
0,1,toy story,animation|children's|comedy
1,2,jumanji,adventure|children's|fantasy
2,3,grumpier old men,comedy|romance
3,4,waiting to exhale,comedy|drama
4,5,father of the bride part ii,comedy


In [894]:
search_movies[search_movies['title']== "gone with the wind"]

,movie_id,title
908,920,gone with the wind


In [895]:
search_movies[search_movies['title']=="toy story"]

,movie_id,title
0,1,toy story


In [896]:
search_movies[search_movies['title']=="jumanji"]

,movie_id,title
1,2,jumanji


In [897]:
search_movies[search_movies['title'].str.contains("lord of the rings, the")]

,movie_id,title
2047,2116,"lord of the rings, the"


In [637]:
# 영화 제목이 겹쳐서 년도를 찾아서 불러왔습니다.
# Titanic : 1721 drama / romance
search_movies[search_movies['title'].str.contains("life is beautiful")] 

,movie_id,title
2255,2324,life is beautiful (la vita � bella)


In [905]:
ratings

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000203,6040,1090,3,956715518
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


#### 선호하는 영화의 이름을 해당하는 movie_id로 바꾸어 준다.
- movie_id 같은 경우 기존에 movie_id로 인덱싱 되어 있었다. 따라서, 새로 넣어주는 movie_id 값도 ratings에 넣기전에 인덱싱을 한 후 넣어준다.

In [916]:
# 영화 제목의 고유한 값에 해당하는 movie_id 값을 인덱싱 해준다. 
unique_movie_title = movies["title"].unique()
unique_movie_id = movies["movie_id"].unique() 
# 고유한 영화를 indexing 하는 코드 
movie_to_idx = {v:k for k,v in zip(unique_movie_id,unique_movie_title)}

In [915]:
#  선호하는 영화의 이름은 꼭 데이터셋에 있는 것과 동일해야지 제대로된 검증이 된다. 
my_favorite = ["gone with the wind" , "life is beautiful (la vita � bella)" ,"toy story" 
               ,"jumanji", "lord of the rings, the"]

# 넣을 영화를 movie_id의 대응하여 인덱싱을 해준다.
indexing_my_favorite = map(lambda x : movie_to_idx[x], my_favorite)

# 'Yong'이라는 user_id가 해당 영화의 선호를 5로 평가 평가한 것입니다.
my_movieList = pd.DataFrame({'user_id': ['Yong']*5, 'movie_id': indexing_my_favorite, 'count':[5]*5})

if not ratings.isin({'user_id':['Yong']})['user_id'].any():  # user_id에 'Yong'이라는 데이터가 없다면
    ratings = ratings.append(my_movieList)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,count,timestamp
1000203,6040,1090,3,956715518.0
1000205,6040,1094,5,956704887.0
1000206,6040,562,5,956704746.0
1000207,6040,1096,4,956715648.0
1000208,6040,1097,4,956715569.0
0,Yong,920,5,NaN
1,Yong,2324,5,NaN
2,Yong,1,5,NaN
3,Yong,2,5,NaN
4,Yong,2116,5,NaN


### 각 컬럼에 대하여 indexing 진행
- 위에서 인덱싱을 한 값을 이용한다.


- movie 데이터의 경우 제대로 인덱싱이 되었는지 확인한다.

In [677]:
# 새로운 user_id가 추가되었으니 고유한 값을 찾는다.
unique_users = ratings["user_id"].unique()

# 고유한 유저를 indexing 하는 코드 
user_to_idx = {v:k for k,v in enumerate(unique_users)}

In [678]:
print(user_to_idx['Yong'])

6039


In [732]:
enumerate_movie_to_idx = {v:k+1 for k,v in enumerate(unique_movie_title)}

In [741]:
unique_movie_title

array(['toy story', 'jumanji', 'grumpier old men', ..., 'tigerland',
       'two family house', 'contender, the'], dtype=object)

In [742]:
unique_movie_id

array([   1,    2,    3, ..., 3950, 3951, 3952])

In [735]:
print("len(unique_movie_title) :", len(unique_movie_title))

len(unique_movie_title) : 3841


In [908]:
len(movie_to_idx)

3841

#### indexing_my_favorite값이 맞게 들어 갔는지 확인

In [909]:
# indexing_my_favorite 해서 넣어줬을때의 해당 영화의 movie_id 값은 1 였다.
movie_to_idx['toy story']

1

In [919]:
# indexing_my_favorite 해서 넣어줬을때의 해당 영화의 movie_id 값은 2 였다.
movie_to_idx['jumanji']

2

In [917]:
# indexing_my_favorite 해서 넣어줬을때의 해당 영화의 movie_id 값은 2324 였다.
movie_to_idx['life is beautiful (la vita � bella)']

2324

In [918]:
# indexing_my_favorite 해서 넣어줬을때의 해당 영화의 movie_id 값은 2116 이다.
movie_to_idx['lord of the rings, the']

2116

- 인덱싱 해서 movie_id에 넣어준 값들을 확인하면 값이 제대로 들어가 있다. 

#### movie_idx값에 맞는 인덱싱을 해주지 않았을 경우
- 아래와 같은 경우 enumerate를 사용해 순서대로 인덱스 값을 매칭하게 만들었다.
- 이런 경우 ratings에 있는 기존의 "gone with the wind"에 해당하는 id가 다른 영화를 가르키게 되버린다.

In [738]:
enumerate_movie_to_idx.get("gone with the wind")

908

#### movie_idx값에 맞는 인덱싱을 해준 경우
- 아래와 같은 경우 movie_id를 고려하여 해당 값으로 인덱싱 한 경우이다.
- 이런 경우 ratings에 있는 기존의 "gone with the wind"에 해당하는 id와 movie에서 뽑은 id가 같은 영화를 가리키게 되어서 올바른 추첨 시스템을 만들 수 있다.

In [910]:
# movie_id 값에 맞게 인덱싱 된것을 확인할 수 있다.
movie_to_idx['gone with the wind']

920

In [688]:
# 컬럼 값 변경 전
ratings['movie_id']

0                                   1193
1                                    661
2                                    914
3                                   3408
4                                   2355
                    ...                 
0                     gone with the wind
1    life is beautiful (la vita � bella)
2                              toy story
3                                jumanji
4                 lord of the rings, the
Name: movie_id, Length: 836483, dtype: object

---
#### Indexing을 통해 데이터 컬럼 내 값을 바꾼다.

In [920]:
# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # ratings['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

user_id column indexing OK!!


In [46]:
ratings[:10]

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


In [815]:
user_to_idx['Yong']

6039

---
movie_id의 경우 선호도 값을 찾은 후 바로 인덱싱을 해주었다.

### CSR matrix 평가 행렬 사용
- CSR Matrix는 Sparse한 matrix에서 0이 아닌 유효한 데이터로 채워지는 데이터의 값과 좌표 정보만으로 구성하여 메모리 사용량을 최소화하면서도 Sparse한 matrix와 동일한 행렬을 표현할 수 있도록 하는 데이터 구조



``` python
csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])  
where data, row_ind and col_ind satisfy the relationship  
a[row_ind[k], col_ind[k]] = data[k]., M,N은 matrix의 shape
```

---
- 유저 X 아이템 평가행렬의 구조이므로 user_id와 movie_id를 이용한다.

In [922]:
# 위에 CSR matrix 구조 설명보고 만들어봅시다.
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)))
# shape를 빼주면 자동으로 맞춰준다.
csr_data

<6040x3953 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

In [923]:
ratings['movie_id'].head(5)

0    1193
1     661
2     914
3    3408
4    2355
Name: movie_id, dtype: int64

###  als_model = AlternatingLeastSquares 모델을 구성하여 훈련
Matrix Factorization 모델을 implicit 패키지를 사용하여 학습한다.

- 이 패키지에 구현된 **als(AlternatingLeastSquares) 모델**을 사용하겠습니다.  <br>**Matrix Factorization**에서 쪼개진 두 Feature Matrix를 한꺼번에 훈련하는 것은 잘 수렴하지 않기 때문에, <br>**한쪽을 고정**시키고 **다른 쪽을 학습**하는 방식을 번갈아 수행하는 **AlternatingLeastSquares** 방식이 효과적인 것으로 알려져 있습니다.

In [924]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

#### AlternatingLeastSquares 클래스의 __init__ 파라미터 
1. **factors** : 유저와 아이템의 벡터를 몇 차원으로 할 것인지 
2. **regularization** : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지 
3. **use_gpu** : GPU를 사용할 것인지 
4. **iterations** : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지

- 1,4를 늘릴수록 학습데이터를 잘 학습하게 되지만 과적합의 우려가 있으니 좋은 값을 찾아야 합니다.

In [989]:
# Implicit AlternatingLeastSquares 모델의 선언
# factors :         500  , 500 차원
# regularization  : 0.01   , 정규화 값 0.01 선언
# use_gpu     :    False   , GPU 사용 안함
# iterations   :    15    , 데이터를 15번 반복해서 학습
als_model = AlternatingLeastSquares(factors=200, regularization=0.01, use_gpu=False, iterations=10, dtype=np.float32)

In [990]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3953x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [991]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/10 [00:00<?, ?it/s]

- 모델이 1) 저의 벡터와 linkin park의 벡터를 어떻게 만들고 있는지와   
  2) 두 벡터를 곱하면 어떤 값이 나오는지 살펴보겠습니다.

In [992]:
my, toy_story = user_to_idx['Yong'], movie_to_idx['toy story']
my_vector, toy_story_vector = als_model.user_factors[my], als_model.item_factors[toy_story]

print('벡터 생성')

벡터 생성


In [993]:
# 50까지만 출력한다.
print("my_vector factor : ", len(my_vector))
my_vector[:50]

my_vector factor :  200


array([ 0.1712223 ,  0.13495584,  0.21607728, -0.33905253, -0.5237309 ,
        0.18975076, -0.27350718, -0.36331016, -0.10748999, -0.54118556,
       -0.21329589, -0.16663527, -0.59613156,  0.83017564,  0.10209788,
        0.08220019,  0.27623445,  0.04218477,  0.5545737 ,  0.9491033 ,
        0.18532953, -0.23850702, -0.0277379 , -0.18780982,  0.10207154,
       -0.6464343 ,  0.39138228, -0.57874024,  0.6738896 , -0.5907967 ,
        0.24202184, -0.4314531 , -0.28341216,  0.7218278 ,  0.15370806,
       -0.04128717,  0.15286718, -0.7295264 ,  0.7401819 , -0.17477477,
       -0.42612854,  0.49021754, -0.49355426, -0.2914742 , -0.40619737,
       -0.45216808,  0.53277683,  0.14190663, -0.40435243, -0.34908462],
      dtype=float32)

In [994]:
# 50까지만 출력한다.
print("toy_story_vector  factor: ",len(toy_story_vector))
toy_story_vector[:50]

toy_story_vector  factor:  200


array([ 0.00591625,  0.02251895,  0.00193943, -0.00459603, -0.0201217 ,
        0.00220069,  0.02561007, -0.02412628,  0.00523313, -0.01856709,
        0.01643783,  0.01763688, -0.01879058,  0.03436386, -0.01539018,
       -0.00126405,  0.02327666,  0.02270289, -0.01752822,  0.02418223,
       -0.00512319, -0.01064339,  0.01162533, -0.00327519,  0.02098029,
       -0.01367118,  0.02124303,  0.0052428 , -0.00210772,  0.0112145 ,
        0.02307367, -0.00957799, -0.00065794,  0.00714944,  0.00119135,
       -0.00545777,  0.00820752, -0.01610825,  0.02046845, -0.00715402,
        0.01867579,  0.0166378 ,  0.00951574, -0.00558572, -0.02106855,
       -0.01623646,  0.0158587 ,  0.0092024 ,  0.01762382,  0.00846494],
      dtype=float32)

In [995]:
# my_vector와 toy_story_vector를 내적하는 코드
np.dot(my_vector, toy_story_vector)

0.7073903

#### toy_story의 대한 선호도가 높게 나왔다.

- "바람과 함께 사라지다 (gone with the wind)"를 넣어서 추가로 확인해 봅시다.

In [996]:
my, gone_wind = user_to_idx['Yong'], movie_to_idx['gone with the wind']
my_vector, gone_wind_vector = als_model.user_factors[my], als_model.item_factors[gone_wind]

print('벡터 생성')

벡터 생성


In [997]:
# my_vector와 Jumanji_vector를 내적하는 코드
# 
np.dot(my_vector, gone_wind_vector)

0.71783626

- 선호도를 나타냈던 영화에 대해 높은 값이 나타난다.

In [1010]:
my, life = user_to_idx['Yong'], movie_to_idx['life is beautiful (la vita � bella)']
my_vector, life_vector = als_model.user_factors[my], als_model.item_factors[life]


In [1011]:
np.dot(my_vector, life_vector)

0.7936193

###  내가 좋아하는 영화와 비슷한 영화 추천 받기
- 모델이 판단하는 내가 선호를 보인 영화의 비슷한 유형을 가진 영화 추천
---
AlternatingLeastSquares 클래스에 구현되어 있는 similar_items 메서드를 통하여 비슷한 영화를 찾습니다. 처음으로는 제가 좋아하는 coldplay로 찾아보겠습니다.

In [998]:
favorite_movie = 'gone with the wind'
movies_id = movie_to_idx[favorite_movie]
similar_movies = als_model.similar_items(movies_id, N=15)
similar_movies

[(920, 1.0),
 (2067, 0.4024211),
 (2594, 0.3521815),
 (1944, 0.3285628),
 (987, 0.3282611),
 (2979, 0.32481512),
 (2982, 0.3227572),
 (437, 0.32162464),
 (1565, 0.32146284),
 (969, 0.3212055),
 (3805, 0.3183625),
 (59, 0.31833473),
 (600, 0.31778145),
 (956, 0.31747356),
 (3218, 0.31722203)]

**(영화 id, 유사도) Tuple** 로 반환하고 있습니다.  
영화의 id를 다시 영화의 이름으로 매핑 시켜 주겠습니다.

In [999]:
#movie_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movies]

['gone with the wind',
 'doctor zhivago',
 'open your eyes (abre los ojos)',
 'from here to eternity',
 'bliss',
 'body shots',
 'guardian, the',
 'cops and robbersons',
 'head above water',
 'african queen, the',
 'knightriders',
 'confessional, the (le confessionnal)',
 'love and a .45',
 'penny serenade',
 'poison']

- '바람과 함께 사라지다'의 장르가 [로맨스,드라마,전쟁]이고 추천 상위 영화들의 장르가 [로맨스,전쟁]에 해당하는걸 보니 모델이 괜찮게 학습 되었네요.

몇 번 더 반복해서 확인하기 위해 위의 코드를 함수로 만들고 확인해보겠습니다.

In [1001]:
def get_similar_artist(movie_title: str):
    movie_id = movie_to_idx[movie_title]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [1003]:
get_similar_artist('titanic')
# 장르에 로맨스,드라마가 포함된 영화들이 주로 나왔습니다.

['titanic',
 'snow day',
 "you've got mail",
 'jerry maguire',
 'sheltering sky, the',
 'walking dead, the',
 'faust',
 'tetsuo ii: body hammer',
 "mr. holland's opus",
 'nina takes a lover']

### 좋아할 만한 영화 추천받기
---
**AlternatingLeastSquares** 클래스에 구현되어 있는 **recommend** 메서드를 통하여 제가 좋아할 만한 영화를 추천받습니다. **filter_already_liked_items** 는 유저가 이미 평가한 아이템은 제외하는 Argument입니다.

In [1004]:
user = user_to_idx['Yong']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(3114, 0.41189498),
 (2355, 0.27623743),
 (34, 0.27190125),
 (2542, 0.19932821),
 (2628, 0.19135176),
 (953, 0.18488841),
 (3489, 0.18354334),
 (2054, 0.18191189),
 (367, 0.17769884),
 (317, 0.15774038),
 (588, 0.15235038),
 (2253, 0.15020035),
 (2804, 0.14974684),
 (653, 0.14703573),
 (249, 0.144313),
 (1242, 0.14169982),
 (919, 0.14125776),
 (1183, 0.13938583),
 (1175, 0.13800839),
 (2366, 0.13758011)]

In [1005]:
[idx_to_movie[i[0]] for i in movie_recommended]

['toy story 2',
 "bug's life, a",
 'babe',
 'lock, stock & two smoking barrels',
 'star wars: episode i - the phantom menace',
 "it's a wonderful life",
 'hook',
 'honey, i shrunk the kids',
 'mask, the',
 'santa clause, the',
 'aladdin',
 'toys',
 'christmas story, a',
 'dragonheart',
 'immortal beloved',
 'glory',
 'wizard of oz, the',
 'english patient, the',
 'delicatessen',
 'king kong']

- 유저에게 영화 추천은 **비슷한 영화 추천**과 마찬가지로 **(영화 id, 유사도) Tuple** 로 반환하고 있습니다.


- toy story를 선호도에 넣었는데 toy story 2가 바로 나왔네요

모델이 **'toy story 2'**를 왜 추천했을까요?  

**AlternatingLeastSquares** 클래스에 구현된 **explain** 메소드를 사용하면 제가 기록을 남긴 데이터 중 이 추천에 기여한 정도를 확인할 수 있습니다.

In [1007]:
rihanna = movie_to_idx['toy story 2']
explain = als_model.explain(user, csr_data, itemid=rihanna)

이 method(explain)는 추천한 콘텐츠의 점수에 기여한 다른 콘텐츠와 기여도(합이 콘텐츠의 점수가 됩니다.)를 반환합니다.   

어떤 영화들이 추천에 얼마나 기여하고 있는 걸까요?

In [1009]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('toy story', 0.3370125760957794),
 ('lord of the rings, the', 0.04709104377614426),
 ('life is beautiful (la vita � bella)', 0.01815034304169759),
 ('jumanji', 0.005904436511437617),
 ('gone with the wind', -0.006566305820911774)]

**toy story**와 **lord of the ring**이 영화 추천에 영향을 주었네요.

- "toy story"가 선호 영화에 있어서 "toy story 2"가 추천 영화에 있는걸 보니 모델이 나쁘지 않게 학습이 되었네요.

---
### 회고


- 다른 프로젝트보다 더 데이터의 준비 과정이나 전처리에 대해 생각해 봤었고 생각한대로 진행하면서 Pandas 문법이 더 익숙해진 것 같습니다. 그리고 추천 시스템을 처음 진행하는데 생각보다 더 재미있어서 즐겁게 프로젝트를 진행한 것 같습니다.


- 데이터 전처리를 진행하면서 느낀점이 있는데 **데이터의 의미**를 생각하지 않은 상태에서 데이터를 전처리 하는 것과 데이터의 의미를 생각하면서 전처리를 진행하는 것의 상당한 차이가 있다는것이 느껴졌습니다. <br><br> 데이터를 제대로 생각하지 않고 전처리를 할 때 모델의 추천 영화가 아예 다른 장르와 느낌의 영화로 추천을 했다면, 다시 데이터의 의미를 생각한 후 전처리를 진행하고 모델을 학습하니 해당 영화와 비슷한 느낌이 나는 영화를 진짜 추천을 해줘서 신기했습니다. 
물론 다른 느낌의 영화도 추천을 해주긴 했지만 그 빈도가 **데이터의 의미**를 생각하지 않고 전처리 했을때 보다 훨씬 줄어들어서 왜 머신러닝이 Data-driven이라고 하는지 전처리를 하면서 많이 알게된 것 같습니다.


- 그리고 프로젝트 주제의 관점으로 봤을때 추천 시스템이 쉽게 접하는 시스템이고 추천 시스템을 제공하는 대표적인 플랫폼인 유튜브나 넷플릭스 등의 시스템에서 어떠한 알고리즘 방식으로 시스템을 구축했는지 궁굼했는데 이번의 얕게나마 그 일부를 알게 돼서 만족할 만한 프로젝트를 진행한 것 같습니다.